In [8]:
import win32com.client
import pandas as pd
import numpy as np
import datetime as dt
import time
from tqdm import tqdm

In [9]:
# 원하는 주식데이터를 가져올 수 있는 대신증권 모듈 선언
instCpCybos = win32com.client.Dispatch('CpUtil.CpCybos')
instCpStockCode = win32com.client.Dispatch("CpUtil.CpStockCode")
StockChart = win32com.client.Dispatch("CpSysDib.StockChart")

In [10]:
# 대신증권 API 연결 확인. 1 출력시 연결 성공, 0 출력시 연결 실패
instCpCybos = win32com.client.Dispatch('CpUtil.CpCybos')
print(instCpCybos.IsConnect)

0


In [3]:
# 대신증권 API 연결 확인. 1 출력시 연결 성공, 0 출력시 연결 실패
def connectCheck():
    print('[시스템]:증권사 연결 시도 중')
    if(instCpCybos.IsConnect == 1):
        print('[시스템]:증권사 연결 완료!')

In [5]:
# 증권시장에서 상장되어 있는 종목의 갯수 출력 -> 새로운 종목이 상장되거나 폐지됨에 따라 값이 달라진다.
def canTradeStockCount():
    print('[시스템]:거래 가능한 종목 개수 : '+str(instCpStockCode.GetCount())+'개 체크')

In [6]:
# 0 -> 종목코드 : 요청할 종목의 종목 코드
# 1 -> 요청구분 : 1은 기간으로 요청, 2는 갯수로 요청
# 2 -> 요청 종료일 : YYYYMMDD 형식
# 3 -> 요청 시작일 : YYYYMMDD 형식
# 4 -> 요청개수 : 요청할 데이터의 개수
# 5 -> 필드 = 0: 날짜, 1:시간, 2:시가, 3: 고가, 4: 저가, 5: 종가, 6: 전일대비, 8: 거래량, 9: 거래대금, 10: 누적체결매도수량
#             11: 누적체결매수수량, 12: 상장주식수, 13: 시가총액, 14: 외국인주문한도수량, 15: 외국인주문가능수량, 16: 외국인현보유수량
#             17: 외국인현보유비율, 18: 수정주가일자, 19: 수정주가비율, 20: 기관순매수, 21: 기관누적순매수, 22: 등락주선, 23: 등락비율
#             24: 예탁금, 25: 주식회전율, 26: 거래성립률, 27: 대비부호
# 6 -> 차트 구분 = D: 일, W: 주, m: 분, T: 틱
# 9 -> 수정주가 = 0: 무수정주가, 1: 수정주가

## 데이터 수집

In [7]:
# 대신증권 기준으로 정렬된 상장된(주식거래가 가능한) 종목을 전부 가져오기
def getStockNameCode():
    print('[시스템]:거래 가능한 종목 리스트 목록 가져오는 중')
    code = [] # 종목코드
    name = [] # 종목명

    for i in range(0, instCpStockCode.GetCount()):
        code.append(instCpStockCode.GetData(0, i))
        name.append(instCpStockCode.GetData(1, i))

    codeNameList = pd.DataFrame({
        '종목코드':code
        , '종목명':name
    })
    # codeNameList
    if(len(codeNameList) == 0):
        print('[시스템]:거래 가능한 종목 리스트 목록 가져온 개수 : '+str(len(codeNameList))+'개')
        print('[시스템]:거래 가능한 종목 리스트 목록 가져오기에 문제가 발생하였습니다.')
    else:
        print('[시스템]:거래 가능한 종목 리스트 목록 가져오기 완료!')

    return codeNameList

In [8]:
# 원하는 종목코드, 리스트를 새로운 데이터프레임에 저장
def jongmokSerach(jongmokSearchList, codeNameList):
    code = []
    name = []
    for names in jongmokSearchList:
        code.append(codeNameList[codeNameList['종목명'] == names]['종목코드'].iloc[0])
        name.append(codeNameList[codeNameList['종목명'] == names]['종목명'].iloc[0])

    selCodeNameList = pd.DataFrame({
        '종목코드':code
        , '종목명':name
    })
    return selCodeNameList

In [9]:
# 주식 데이터 가져오기
def getjusik(jusikCode, jusikName, start, end):
    code = [] # 종목코드
    name = [] # 종목명
    c0 = []   # 날짜
    c1 = []   # 시간
    c2 = []   # 시가
    c3 = []   # 고가
    c4 = []   # 저가
    c5 = []   # 종가
    c8 = []   # 거래량
    # c9 = []   # 거래대금
    # c10 = []  # 누적체결매도수량
    # c11 = []  # 누적체결매수수량

    StockChart.SetInputValue(0, jusikCode)
    StockChart.SetInputValue(1, ord('1'))
    StockChart.SetInputValue(2, end)
    StockChart.SetInputValue(3, start)
    StockChart.SetInputValue(4, 2000)
    StockChart.SetInputValue(5, (0,1,2,3,4,5,8,9,10,11,12,13))
    StockChart.SetInputValue(6, ord('m'))
    StockChart.SetInputValue(9, ord('1'))

    StockChart.BlockRequest()

    dataFea = StockChart.GetHeaderValue(2)
    numData = StockChart.GetHeaderValue(3)
                 
    for i in range(numData):
        code.append(jusikCode)
        name.append(jusikName)
        c0.append(str(StockChart.GetDataValue(0, i)))
        c1.append(str(StockChart.GetDataValue(1, i)))
        c2.append(StockChart.GetDataValue(2, i))
        c3.append(StockChart.GetDataValue(3, i))
        c4.append(StockChart.GetDataValue(4, i))
        c5.append(StockChart.GetDataValue(5, i))
        c8.append(StockChart.GetDataValue(8, i))
        # c9.append(StockChart.GetDataValue(9, i))
        # c10.append(StockChart.GetDataValue(10, i))
        # c11.append(StockChart.GetDataValue(11, i))
    dataDf = pd.DataFrame()
    dataDf['종목코드'] = code
    dataDf['종목명'] = name
    dataDf['날짜'] = c0
    dataDf['시간'] = c1
    dataDf['시가'] = c2
    dataDf['고가'] = c3
    dataDf['저가'] = c4
    dataDf['종가'] = c5
    dataDf['거래량'] = c8
    # dataDf['거래대금'] = c9
    # dataDf['누적체결매도수량'] = c10
    # dataDf['누적체결매수수량'] = c11

    return dataDf

In [10]:
# 완성된 csv파일 출력하기
def aaa(stockNameList):
    return pd.read_csv('./stockdata/'+stockNameList[0]+' 주식 데이터.csv', encoding='euc-kr')

In [11]:
# 수신 받은 주식데이터 정렬하기
def reverse(stockNameList):
    print('[시스템]:수신받은 데이터 날짜, 시간순으로 정렬 작업중')
    for i in range(len(stockNameList)):
        data1 = pd.read_csv('./stockdata/'+stockNameList[i]+' 주식 데이터.csv', encoding='euc-kr')

        data1.sort_values(['날짜','시간'], inplace=True)
        data1.reset_index(inplace=True)
        data1.drop('index', axis=1, inplace=True)
        data1.to_csv('./stockdata/'+stockNameList[i]+' 주식 데이터.csv', encoding='euc-kr', index=False)
    print('[시스템]:정렬 완료!')

In [14]:
# 주식데이터를 받기 위해 실행

stockNameList = ['루닛'] # 이곳에 수신 받고 싶은 종목 이름을 정확히 입력해주세요. 여러개 가능.

connectCheck()
canTradeStockCount()
codeNameList = getStockNameCode()
selCodeNameList = jongmokSerach(stockNameList, codeNameList)

a = dt.datetime(2021,7,9) # 수신 받고 싶은 원하는 시작 날짜를 입력하세요.
b = dt.datetime(2023,7,19) # 수신 받고 싶은 원하는 종료 날짜를 입력하세요.
number = b-a

for j in range(0,selCodeNameList.shape[0]):
    print('[시스템]:'+str(a.strftime('%Y%m%d'))+' ~ '+str(b.strftime('%Y%m%d'))+' 사이에 거래된 '+str(selCodeNameList['종목명'].iloc[j])+' 종목 주가 데이터 가져오는 중')
    frame = []
    for i in tqdm(range(0, number.days, 2)):
        c = a + dt.timedelta(days=i)
        d = a + dt.timedelta(days=i+1)
        data = getjusik(selCodeNameList['종목코드'].iloc[j], selCodeNameList['종목명'].iloc[j], c.strftime('%Y%m%d'), d.strftime('%Y%m%d'))
        frame.append(data)
        time.sleep(0.3)
    hapdata = pd.concat(frame)
    hapdata.to_csv('./stock/'+selCodeNameList['종목명'].iloc[j]+' 주식 데이터.csv', encoding='euc-kr', index=False)
    print('[시스템]:'+selCodeNameList['종목명'].iloc[j]+' 종목의 주식시세 데이터 요청 수신성공 및 csv로 저장 성공!')

reverse(stockNameList)
print('[시스템]:수신요청 종료')
aaa(stockNameList)

[시스템]:증권사 연결 시도 중
[시스템]:증권사 연결 완료!
[시스템]:거래 가능한 종목 개수 : 3997개 체크
[시스템]:거래 가능한 종목 리스트 목록 가져오는 중
[시스템]:거래 가능한 종목 리스트 목록 가져오기 완료!
[시스템]:20210709 ~ 20230719 사이에 거래된 루닛 종목 주가 데이터 가져오는 중


100%|██████████| 370/370 [02:10<00:00,  2.83it/s]


[시스템]:루닛 종목의 주식시세 데이터 요청 수신성공 및 csv로 저장 성공!
[시스템]:수신받은 데이터 날짜, 시간순으로 정렬 작업중


FileNotFoundError: [Errno 2] No such file or directory: './stockdata/루닛 주식 데이터.csv'